In [1]:
# Importamos las librerías.
from bs4 import BeautifulSoup
import requests

# Scraping

In [2]:
# 1. Obtenemos el HTML
URL_BASE = 'https://scrapepark.org/courses/spanish/'
pedido_obtenido = requests.get(URL_BASE)
html_obtenido = pedido_obtenido.text

# 2. "Parsear" ese HTML
soup = BeautifulSoup(html_obtenido, "html.parser")

### El método **find()**

Permite quedarnos con la información asociada a una etiqueta de HTML

In [3]:
primer_h2 = soup.find('h2')
print(primer_h2)

<h2>¿Por qué comprar con nosotros?</h2>


In [4]:
# Solo el texto.
print(primer_h2.text)

# Equivalente a: 
print(soup.h2.text)

¿Por qué comprar con nosotros?
¿Por qué comprar con nosotros?


### El método **find()**

Busca **TODOS** los elementos de la página con esa etiqueta y devuelve una "lista" que los contiene (en realidad devuelve un objeto de la clase bs4.element.ResultSet).

In [5]:
h2_todos = soup.find_all('h2')
print(h2_todos)

[<h2>¿Por qué comprar con nosotros?</h2>, <h2>
                  #Novedades
                </h2>, <h2>
            Nuestros <span>productos</span>
</h2>, <h2>
            Testimonios de clientes
          </h2>, <h2 class="heading-container">
          Tabla de precios
        </h2>]


In [6]:
# Si usamos el parámetro limit = 1, emulamos al metodo find
h2_uno_solo = soup.find_all('h2', limit = 1)
print(h2_uno_solo)

[<h2>¿Por qué comprar con nosotros?</h2>]


In [7]:
# Podemos iterar sobre el objeto.
for seccion in h2_todos:
    print(seccion.text)

¿Por qué comprar con nosotros?

                  #Novedades
                

            Nuestros productos


            Testimonios de clientes
          

          Tabla de precios
        


In [8]:
# get_text() para más funcionalidades.
for seccion in h2_todos:
    print(seccion.get_text(strip = True)) # Limpiamos los espacios en blanco.

¿Por qué comprar con nosotros?
#Novedades
Nuestrosproductos
Testimonios de clientes
Tabla de precios


### Utilizando atributos de la página.

In [9]:
# Clase.
divs = soup.find_all('div', class_ = "heading-container heading-center")

for div in divs:
    print(div)
    print('\n')

<div class="heading-container heading-center" id="acerca">
<h2>¿Por qué comprar con nosotros?</h2>
</div>


<div class="heading-container heading-center" id="productos">
<h2>
            Nuestros <span>productos</span>
</h2>
</div>


<div class="heading-container heading-center">
<h3>Suscríbete para obtener descuentos y ofertas</h3>
</div>


<div class="heading-container heading-center">
<h2>
            Testimonios de clientes
          </h2>
</div>




In [10]:
# Todas las etiquetas que tengan el atributo "src".
src_todos = soup.find_all(src = True)

for elemento in src_todos:
    if elemento['src'].endswith('.jpg'):
        print(elemento)

<img alt="Parque de patinaje" src="images/slider-bg.jpg"/>
<img alt="Patineta 2" src="images/p2.jpg"/>


Bajamos todas las imágenes.

In [11]:
url_imagenes = []

for i, imagen in enumerate(src_todos):
    if imagen['src'].endswith('png'):
        print(imagen['src'])
        r = requests.get(f'https://scrapepark.org/courses/spanish/{imagen['src']}')

        with open(f'imagen_{i}.png', 'wb') as f:
            f.write(r.content)

images/arrival-bg-store.png
images/p1.png
images/p3.png
images/p4.png
images/p5.png
images/p6.png
images/p7.png
images/p8.png
images/p9.png
images/p10.png
images/p11.png
images/p12.png
images/client-one.png
images/client-two.png
images/client-three.png
./images/freecodecamp-logo.png


In [12]:
# Información de tablas.
URL_BASE = 'https://scrapepark.org/courses/spanish'
URL_TABLA = soup.find_all('iframe')[0]['src']

requests_tabla = requests.get(f'{URL_BASE}/{URL_TABLA}')

html_tabla = requests_tabla.text
soup_tabla = BeautifulSoup(html_tabla, 'html.parser')
soup_tabla.find('tabla')

productos_faltantes = soup_tabla.find_all(['th', 'td'], attrs = {'style': 'color: red;'})
productos_faltantes = [talle.text for talle in productos_faltantes]

print(productos_faltantes)

['Longboard', '$80', '$85', '$90', '$62', '$150']


In [ ]:
asd = 3